In [31]:
from nba_api.stats.endpoints import playercareerstats
import matplotlib.pyplot as plt
from nba_api.stats.static import players
import seaborn as sns
import numpy as np
import pandas as pd
from json import loads, dumps
from nba_api.stats.static import players
from nba_api.stats.endpoints import commonplayerinfo, playercareerstats


pd.set_option("display.max_columns", None)

In [46]:
from nba_api.stats.static import teams
from nba_api.stats.endpoints import commonteamroster
from nba_api.stats.endpoints import teamgamelog
from nba_api.stats.endpoints import commonteamroster
from datetime import datetime, timezone
from dateutil import parser
from nba_api.live.nba.endpoints import scoreboard
from nba_api.stats.endpoints import playerdashboardbyyearoveryear
from nba_api.stats.endpoints import playergamelog


In [47]:
def get_current_season():
    """Get current NBA season in the format 'YYYY-YY'"""
    today = datetime.now()
    year = today.year
    month = today.month
    
    # NBA season typically starts in October
    # If current month is before October, we're in the second half of the season
    if month < 10:
        season_start = year - 1
    else:
        season_start = year
    
    season_end = str(season_start + 1)[-2:]  # Get last 2 digits
    return f"{season_start}-{season_end}"

In [ ]:
class player():
    def __init__(self, player_name):
        self.player_dict = players.find_players_by_full_name(player_name)[0]
        self.player_id = self.player_dict['id']
        self.career = playercareerstats.PlayerCareerStats(player_id=self.player_id)
        self.career_df = self.career.get_data_frames()[0]
        self.current_season = get_current_season()

    def get_player_info(self):
        player_info = commonplayerinfo.CommonPlayerInfo(player_id=self.player_id)
        info_df = player_info.get_data_frames()[0]
        return info_df
    
    def get_player_latest_performance(self):
        game_log = playergamelog.PlayerGameLog(
            player_id=self.player_id,
            season=self.current_season
        )
        df = game_log.get_data_frames()[0]

        if df.empty:
            return "Player is not active this season or has not played any games."
        return df.iloc[0]

    def get_alltime_player_stats(self):
        return self.career_df

    def get_current_season_stats(self):
        current_season = self.current_season
        player_dashboard = playerdashboardbyyearoveryear.PlayerDashboardByYearOverYear(
            player_id=self.player_id,
            season=current_season
        )
        current_stats = player_dashboard.get_data_frames()[0]
        current_stats = player_dashboard.get_data_frames()[0]
        current_stats = current_stats[["GP", "MIN", "FG_PCT", "FG3_PCT", "FT_PCT", "REB", 
                                    "AST", "PTS", "BLK", "PLUS_MINUS"]]
        current_stats["PPG"] = round(current_stats["PTS"] / current_stats["GP"], 2)
        current_stats["APG"] = round(current_stats["AST"] / current_stats["GP"], 2)
        current_stats["RPG"] = round(current_stats["REB"] / current_stats["GP"], 2)
        current_stats["BPG"] = round(current_stats["BLK"] / current_stats["GP"], 2)
        current_stats["MPG"] = round(current_stats["MIN"] / current_stats["GP"], 2) 
        current_stats
        return current_stats


    def plot_stat_over_career(self, stat):
        print("got df")
        if stat not in self.career_df.columns:
            raise ValueError(f"Stat '{stat}' not found in career data.")
        
        
        plt.figure(figsize=(10, 6))
        sns.lineplot(data=self.career_df, x='SEASON_ID', y=stat, marker='o')
        plt.title(f"{self.player_dict['full_name']}'s {stat} Over Career")
        plt.xlabel('Season')
        plt.ylabel(stat)
        plt.xticks(rotation=45)
        plt.grid() 
        plt.show()

In [ ]:
player('Anthony Edwards').get_player_latest_performance()

SEASON_ID                 22025
Player_ID               1630162
Game_ID              0022500044
GAME_DATE          Nov 14, 2025
MATCHUP             MIN vs. SAC
WL                            W
MIN                          36
FGM                           9
FGA                          20
FG_PCT                     0.45
FG3M                          1
FG3A                          7
FG3_PCT                   0.143
FTM                          11
FTA                          12
FT_PCT                    0.917
OREB                          0
DREB                          4
REB                           4
AST                           4
STL                           2
BLK                           0
TOV                           3
PF                            2
PTS                          30
PLUS_MINUS                    9
VIDEO_AVAILABLE               1
Name: 0, dtype: object

In [41]:
first_game

PLAYER_ID                201937
SEASON_ID               2011-12
LEAGUE_ID                    00
TEAM_ID              1610612750
TEAM_ABBREVIATION           MIN
PLAYER_AGE                 21.0
GP                           41
GS                           31
MIN                        1404
FGM                         139
FGA                         389
FG_PCT                    0.357
FG3M                         32
FG3A                         94
FG3_PCT                    0.34
FTM                         126
FTA                         157
FT_PCT                    0.803
OREB                         21
DREB                        150
REB                         171
AST                         336
STL                          91
BLK                           8
TOV                         131
PF                           97
PTS                         436
Name: 0, dtype: object

In [42]:
last_game

,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,FGA,FG_PCT,FG3M,FG3A,FG3_PCT,FTM,FTA,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,201937,2011-12,00,1610612750,MIN,21.0,41,31,1404,139,389,0.357,32,94,0.340,126,157,0.803,21,150,171,336,91,8,131,97,436
1,201937,2012-13,00,1610612750,MIN,22.0,57,47,1691,185,514,0.360,27,92,0.293,211,264,0.799,44,185,229,418,137,5,172,143,608
2,201937,2013-14,00,1610612750,MIN,23.0,82,82,2638,255,670,0.381,44,133,0.331,227,283,0.802,61,281,342,704,191,11,221,218,781
3,201937,2014-15,00,1610612750,MIN,24.0,22,22,692,78,219,0.356,13,51,0.255,57,71,0.803,19,106,125,193,38,1,64,59,226
4,201937,2015-16,00,1610612750,MIN,25.0,76,76,2323,219,586,0.374,62,190,0.326,266,314,0.847,40,286,326,658,162,10,193,201,766
5,201937,2016-17,00,1610612750,MIN,26.0,75,75,2469,261,650,0.402,60,196,0.306,254,285,0.891,68,237,305,682,128,10,195,202,836
6,201937,2017-18,00,1610612762,UTA,27.0,77,77,2254,346,827,0.418,96,273,0.352,220,254,0.866,44,307,351,410,121,11,207,206,1008
7,201937,2018-19,00,1610612762,UTA,28.0,68,67,1899,295,730,0.404,79,254,0.311,195,228,0.855,33,210,243,416,91,10,180,180,864
8,201937,2019-20,00,1610612756,PHX,29.0,65,65,2016,287,692,0.415,78,216,0.361,195,226,0.863,46,258,304,570,94,11,173,167,847
9,201937,2020-21,00,1610612750,MIN,30.0,68,51,1772,193,498,0.388,66,214,0.308,130,150,0.867,28,195,223,433,98,4,111,139,582


In [25]:
from datetime import datetime

def get_current_season():
    """Get current NBA season in the format 'YYYY-YY'"""
    today = datetime.now()
    year = today.year
    month = today.month
    
    # NBA season typically starts in October
    # If current month is before October, we're in the second half of the season
    if month < 10:
        season_start = year - 1
    else:
        season_start = year
    
    season_end = str(season_start + 1)[-2:]  # Get last 2 digits
    return f"{season_start}-{season_end}"

# Usage
lebron_player = players.find_players_by_full_name('Deandre Ayton')[0]
player_id = lebron_player['id']

current_season = get_current_season()  # Returns '2024-25' for now
player_dashboard = playerdashboardbyyearoveryear.PlayerDashboardByYearOverYear(
    player_id=player_id,
    season=current_season
)
current_stats = player_dashboard.get_data_frames()[0]
current_stats = current_stats[["GP", "MIN", "FG_PCT", "FG3_PCT", "FT_PCT", "REB", 
                               "AST", "PTS", "BLK", "PLUS_MINUS"]]
current_stats["PPG"] = round(current_stats["PTS"] / current_stats["GP"], 2)
current_stats["APG"] = round(current_stats["AST"] / current_stats["GP"], 2)
current_stats["RPG"] = round(current_stats["REB"] / current_stats["GP"], 2)
current_stats["BPG"] = round(current_stats["BLK"] / current_stats["GP"], 2)
current_stats["MPG"] = round(current_stats["MIN"] / current_stats["GP"], 2)
current_stats

,GP,MIN,FG_PCT,FG3_PCT,FT_PCT,REB,AST,PTS,BLK,PLUS_MINUS,PPG,APG,RPG,BPG,MPG
0,13,394.616666,0.692,0.0,0.619,109,12,211,12,-3,16.23,0.92,8.38,0.92,30.36


In [ ]:
from nba_api.stats.static import players
from nba_api.stats.endpoints import commonplayerinfo, playercareerstats

# Get all players (returns a list of dictionaries with basic info)
all_players = players.get_players()

# Find a specific player by name
lebron_player = players.find_players_by_full_name('LeBron James')[0]  # Returns list, get first match
# OR find by last name
players_named_james = players.find_players_by_last_name('James')
# OR find by first name
players_named_lebron = players.find_players_by_first_name('LeBron')

# Get player ID
player_id = lebron_player['id']

# Get detailed player information (non-statistical)
player_info = commonplayerinfo.CommonPlayerInfo(player_id=player_id)
player_info_dict = player_info.get_normalized_dict()
player_common_info = player_info_dict['CommonPlayerInfo'][0]

# This gives you access to:
# - Name, height, weight, etc.
# - Birth date, country
# - Draft information
# - Position, jersey number
# - Team information

In [ ]:
lebron_df =  player_info.get_data_frames()[0]

In [9]:
lebron_df.columns

Index(['PERSON_ID', 'FIRST_NAME', 'LAST_NAME', 'DISPLAY_FIRST_LAST',
       'DISPLAY_LAST_COMMA_FIRST', 'DISPLAY_FI_LAST', 'PLAYER_SLUG',
       'BIRTHDATE', 'SCHOOL', 'COUNTRY', 'LAST_AFFILIATION', 'HEIGHT',
       'WEIGHT', 'SEASON_EXP', 'JERSEY', 'POSITION', 'ROSTERSTATUS',
       'GAMES_PLAYED_CURRENT_SEASON_FLAG', 'TEAM_ID', 'TEAM_NAME',
       'TEAM_ABBREVIATION', 'TEAM_CODE', 'TEAM_CITY', 'PLAYERCODE',
       'FROM_YEAR', 'TO_YEAR', 'DLEAGUE_FLAG', 'NBA_FLAG', 'GAMES_PLAYED_FLAG',
       'DRAFT_YEAR', 'DRAFT_ROUND', 'DRAFT_NUMBER', 'GREATEST_75_FLAG'],
      dtype='object')

In [10]:
columns = lebron_df.columns.tolist()

In [11]:
print(columns)

['PERSON_ID', 'FIRST_NAME', 'LAST_NAME', 'DISPLAY_FIRST_LAST', 'DISPLAY_LAST_COMMA_FIRST', 'DISPLAY_FI_LAST', 'PLAYER_SLUG', 'BIRTHDATE', 'SCHOOL', 'COUNTRY', 'LAST_AFFILIATION', 'HEIGHT', 'WEIGHT', 'SEASON_EXP', 'JERSEY', 'POSITION', 'ROSTERSTATUS', 'GAMES_PLAYED_CURRENT_SEASON_FLAG', 'TEAM_ID', 'TEAM_NAME', 'TEAM_ABBREVIATION', 'TEAM_CODE', 'TEAM_CITY', 'PLAYERCODE', 'FROM_YEAR', 'TO_YEAR', 'DLEAGUE_FLAG', 'NBA_FLAG', 'GAMES_PLAYED_FLAG', 'DRAFT_YEAR', 'DRAFT_ROUND', 'DRAFT_NUMBER', 'GREATEST_75_FLAG']


In [12]:
lebron_df.columns

Index(['PERSON_ID', 'FIRST_NAME', 'LAST_NAME', 'DISPLAY_FIRST_LAST',
       'DISPLAY_LAST_COMMA_FIRST', 'DISPLAY_FI_LAST', 'PLAYER_SLUG',
       'BIRTHDATE', 'SCHOOL', 'COUNTRY', 'LAST_AFFILIATION', 'HEIGHT',
       'WEIGHT', 'SEASON_EXP', 'JERSEY', 'POSITION', 'ROSTERSTATUS',
       'GAMES_PLAYED_CURRENT_SEASON_FLAG', 'TEAM_ID', 'TEAM_NAME',
       'TEAM_ABBREVIATION', 'TEAM_CODE', 'TEAM_CITY', 'PLAYERCODE',
       'FROM_YEAR', 'TO_YEAR', 'DLEAGUE_FLAG', 'NBA_FLAG', 'GAMES_PLAYED_FLAG',
       'DRAFT_YEAR', 'DRAFT_ROUND', 'DRAFT_NUMBER', 'GREATEST_75_FLAG'],
      dtype='object')

In [13]:
lebron_df

,PERSON_ID,FIRST_NAME,LAST_NAME,DISPLAY_FIRST_LAST,DISPLAY_LAST_COMMA_FIRST,DISPLAY_FI_LAST,PLAYER_SLUG,BIRTHDATE,SCHOOL,COUNTRY,...,PLAYERCODE,FROM_YEAR,TO_YEAR,DLEAGUE_FLAG,NBA_FLAG,GAMES_PLAYED_FLAG,DRAFT_YEAR,DRAFT_ROUND,DRAFT_NUMBER,GREATEST_75_FLAG
0,2544,LeBron,James,LeBron James,"James, LeBron",L. James,lebron-james,1984-12-30T00:00:00,St. Vincent-St. Mary HS (OH),USA,...,lebron_james,2003,2025,N,Y,Y,2003,1,1,Y


In [14]:
from nba_api.stats.endpoints import playercareerstats
from nba_api.stats.static import players
import pandas as pd

# Buscar jugador por nombre
player_dict = players.find_players_by_full_name("LeBron James")[0]
player_id = player_dict['id']

# Obtener estadísticas de carrera
career = playercareerstats.PlayerCareerStats(player_id=player_id)
df = career.get_data_frames()[0]

# Mostrar estadísticas por temporada
df


,PLAYER_ID,SEASON_ID,LEAGUE_ID,TEAM_ID,TEAM_ABBREVIATION,PLAYER_AGE,GP,GS,MIN,FGM,...,FT_PCT,OREB,DREB,REB,AST,STL,BLK,TOV,PF,PTS
0,2544,2003-04,00,1610612739,CLE,19.0,79,79,3120,622,...,0.754,99,333,432,465,130,58,273,149,1654
1,2544,2004-05,00,1610612739,CLE,20.0,80,80,3388,795,...,0.750,111,477,588,577,177,52,262,146,2175
2,2544,2005-06,00,1610612739,CLE,21.0,79,79,3361,875,...,0.738,75,481,556,521,123,66,260,181,2478
3,2544,2006-07,00,1610612739,CLE,22.0,78,78,3190,772,...,0.698,83,443,526,470,125,55,250,171,2132
4,2544,2007-08,00,1610612739,CLE,23.0,75,74,3027,794,...,0.712,133,459,592,539,138,81,255,165,2250
5,2544,2008-09,00,1610612739,CLE,24.0,81,81,3054,789,...,0.780,106,507,613,587,137,93,241,139,2304
6,2544,2009-10,00,1610612739,CLE,25.0,76,76,2966,768,...,0.767,71,483,554,651,125,77,261,119,2258
7,2544,2010-11,00,1610612748,MIA,26.0,79,79,3063,758,...,0.759,80,510,590,554,124,50,284,163,2111
8,2544,2011-12,00,1610612748,MIA,27.0,62,62,2326,621,...,0.771,94,398,492,387,115,50,213,96,1683
9,2544,2012-13,00,1610612748,MIA,28.0,76,76,2877,765,...,0.753,97,513,610,551,129,67,226,110,2036


In [15]:
df.columns

Index(['PLAYER_ID', 'SEASON_ID', 'LEAGUE_ID', 'TEAM_ID', 'TEAM_ABBREVIATION',
       'PLAYER_AGE', 'GP', 'GS', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M', 'FG3A',
       'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST', 'STL',
       'BLK', 'TOV', 'PF', 'PTS'],
      dtype='object')

In [16]:
def get_team_roster_per_season(teamAbbreviation:str, season:str) -> pd.DataFrame:
    team_details = teams.find_team_by_abbreviation(teamAbbreviation)
    team_id = team_details["id"]
    roster = commonteamroster.CommonTeamRoster(team_id=team_id, season=season)
    roster_data = roster.get_data_frames()[0]
    return roster_data

In [17]:
LAL_roster = get_team_roster_per_season("LAL",'2024-25')["PLAYER_ID"].tolist()

In [18]:
LAL_roster

[1629020,
 1642261,
 1629216,
 1642355,
 1631132,
 1629637,
 1628467,
 1630559,
 1627827,
 1629003,
 2544,
 203458,
 1629060,
 1630692,
 1641998,
 1629029,
 202693]

In [19]:
players = [2544, 203458]
columns = ['PERSON_ID', 'FIRST_NAME', 'LAST_NAME', 'DISPLAY_FIRST_LAST', 'DISPLAY_LAST_COMMA_FIRST', 'DISPLAY_FI_LAST', 'PLAYER_SLUG', 'BIRTHDATE', 'SCHOOL', 'COUNTRY', 'LAST_AFFILIATION', 'HEIGHT', 'WEIGHT', 'SEASON_EXP', 'JERSEY', 'POSITION', 'ROSTERSTATUS', 'GAMES_PLAYED_CURRENT_SEASON_FLAG', 'TEAM_ID', 'TEAM_NAME', 'TEAM_ABBREVIATION', 'TEAM_CODE', 'TEAM_CITY', 'PLAYERCODE', 'FROM_YEAR', 'TO_YEAR', 'DLEAGUE_FLAG', 'NBA_FLAG', 'GAMES_PLAYED_FLAG', 'DRAFT_YEAR', 'DRAFT_ROUND', 'DRAFT_NUMBER', 'GREATEST_75_FLAG']
empty_df = pd.DataFrame(columns=columns)


for player_id in players:
    player_info = commonplayerinfo.CommonPlayerInfo(player_id=player_id)
    player_info_dict = player_info.get_normalized_dict()
    player_common_info = player_info_dict['CommonPlayerInfo'][0]

    empty_df.loc[len(empty_df)] = player_common_info


In [20]:
empty_df

,PERSON_ID,FIRST_NAME,LAST_NAME,DISPLAY_FIRST_LAST,DISPLAY_LAST_COMMA_FIRST,DISPLAY_FI_LAST,PLAYER_SLUG,BIRTHDATE,SCHOOL,COUNTRY,...,PLAYERCODE,FROM_YEAR,TO_YEAR,DLEAGUE_FLAG,NBA_FLAG,GAMES_PLAYED_FLAG,DRAFT_YEAR,DRAFT_ROUND,DRAFT_NUMBER,GREATEST_75_FLAG
0,2544,LeBron,James,LeBron James,"James, LeBron",L. James,lebron-james,1984-12-30T00:00:00,St. Vincent-St. Mary HS (OH),USA,...,lebron_james,2003,2025,N,Y,Y,2003,1,1,Y
1,203458,Alex,Len,Alex Len,"Len, Alex",A. Len,alex-len,1993-06-16T00:00:00,Maryland,Ukraine,...,alex_len,2013,2024,N,Y,Y,2013,1,5,N


In [21]:
import time 
def get_roster_data(roster):
    columns = ['PERSON_ID', 'FIRST_NAME', 'LAST_NAME', 'DISPLAY_FIRST_LAST', 'DISPLAY_LAST_COMMA_FIRST', 'DISPLAY_FI_LAST', 'PLAYER_SLUG', 'BIRTHDATE', 'SCHOOL', 'COUNTRY', 'LAST_AFFILIATION', 'HEIGHT', 'WEIGHT', 'SEASON_EXP', 'JERSEY', 'POSITION', 'ROSTERSTATUS', 'GAMES_PLAYED_CURRENT_SEASON_FLAG', 'TEAM_ID', 'TEAM_NAME', 'TEAM_ABBREVIATION', 'TEAM_CODE', 'TEAM_CITY', 'PLAYERCODE', 'FROM_YEAR', 'TO_YEAR', 'DLEAGUE_FLAG', 'NBA_FLAG', 'GAMES_PLAYED_FLAG', 'DRAFT_YEAR', 'DRAFT_ROUND', 'DRAFT_NUMBER', 'GREATEST_75_FLAG']
    empty_df = pd.DataFrame(columns=columns)
    
    for player_id in roster:
        player_info = commonplayerinfo.CommonPlayerInfo(player_id=player_id)
        player_info_dict = player_info.get_normalized_dict()
        player_common_info = player_info_dict['CommonPlayerInfo'][0]
    
        empty_df.loc[len(empty_df)] = player_common_info

        time.sleep(0.8)
        

    return empty_df

In [22]:
LAL_current_roster = get_roster_data(LAL_roster)

In [23]:
LAL_current_roster.columns

Index(['PERSON_ID', 'FIRST_NAME', 'LAST_NAME', 'DISPLAY_FIRST_LAST',
       'DISPLAY_LAST_COMMA_FIRST', 'DISPLAY_FI_LAST', 'PLAYER_SLUG',
       'BIRTHDATE', 'SCHOOL', 'COUNTRY', 'LAST_AFFILIATION', 'HEIGHT',
       'WEIGHT', 'SEASON_EXP', 'JERSEY', 'POSITION', 'ROSTERSTATUS',
       'GAMES_PLAYED_CURRENT_SEASON_FLAG', 'TEAM_ID', 'TEAM_NAME',
       'TEAM_ABBREVIATION', 'TEAM_CODE', 'TEAM_CITY', 'PLAYERCODE',
       'FROM_YEAR', 'TO_YEAR', 'DLEAGUE_FLAG', 'NBA_FLAG', 'GAMES_PLAYED_FLAG',
       'DRAFT_YEAR', 'DRAFT_ROUND', 'DRAFT_NUMBER', 'GREATEST_75_FLAG'],
      dtype='object')

In [24]:
LAL_current_roster

,PERSON_ID,FIRST_NAME,LAST_NAME,DISPLAY_FIRST_LAST,DISPLAY_LAST_COMMA_FIRST,DISPLAY_FI_LAST,PLAYER_SLUG,BIRTHDATE,SCHOOL,COUNTRY,...,PLAYERCODE,FROM_YEAR,TO_YEAR,DLEAGUE_FLAG,NBA_FLAG,GAMES_PLAYED_FLAG,DRAFT_YEAR,DRAFT_ROUND,DRAFT_NUMBER,GREATEST_75_FLAG
0,1629020,Jarred,Vanderbilt,Jarred Vanderbilt,"Vanderbilt, Jarred",J. Vanderbilt,jarred-vanderbilt,1999-04-03T00:00:00,Kentucky,USA,...,jarred_vanderbilt,2018,2025,Y,Y,Y,2018,2,41,N
1,1642261,Dalton,Knecht,Dalton Knecht,"Knecht, Dalton",D. Knecht,dalton-knecht,2001-04-19T00:00:00,Tennessee,USA,...,dalton_knecht,2024,2025,N,Y,Y,2024,1,17,N
2,1629216,Gabe,Vincent,Gabe Vincent,"Vincent, Gabe",G. Vincent,gabe-vincent,1996-06-14T00:00:00,California-Santa Barbara,USA,...,gabe_vincent,2019,2025,Y,Y,Y,Undrafted,Undrafted,Undrafted,N
3,1642355,Bronny,James,Bronny James,"James, Bronny",B. James,bronny-james,2004-10-06T00:00:00,Southern California,USA,...,tmp_bronny_james,2024,2025,Y,Y,Y,2024,2,55,N
4,1631132,Christian,Koloko,Christian Koloko,"Koloko, Christian",C. Koloko,christian-koloko,2000-06-20T00:00:00,Arizona,Cameroon,...,christian_koloko,2022,2025,Y,Y,Y,2022,2,33,N
5,1629637,Jaxson,Hayes,Jaxson Hayes,"Hayes, Jaxson",J. Hayes,jaxson-hayes,2000-05-23T00:00:00,Texas,USA,...,jaxson_hayes,2019,2025,Y,Y,Y,2019,1,8,N
6,1628467,Maxi,Kleber,Maxi Kleber,"Kleber, Maxi",M. Kleber,maxi-kleber,1992-01-29T00:00:00,Bayern Munich,Germany,...,maxi_kleber,2017,2025,N,Y,Y,Undrafted,Undrafted,Undrafted,N
7,1630559,Austin,Reaves,Austin Reaves,"Reaves, Austin",A. Reaves,austin-reaves,1998-05-29T00:00:00,Oklahoma,USA,...,austin_reaves,2021,2025,N,Y,Y,Undrafted,Undrafted,Undrafted,N
8,1627827,Dorian,Finney-Smith,Dorian Finney-Smith,"Finney-Smith, Dorian",D. Finney-Smith,dorian-finney-smith,1993-05-04T00:00:00,Florida,USA,...,dorian_finney-smith,2016,2025,N,Y,Y,Undrafted,Undrafted,Undrafted,N
9,1629003,Shake,Milton,Shake Milton,"Milton, Shake",S. Milton,shake-milton,1996-09-26T00:00:00,Southern Methodist,USA,...,shake_milton,2018,2024,Y,Y,Y,2018,2,54,N


In [25]:
LAL_current_roster['PLAYERCODE']

0       jarred_vanderbilt
1           dalton_knecht
2            gabe_vincent
3        tmp_bronny_james
4        christian_koloko
5            jaxson_hayes
6             maxi_kleber
7           austin_reaves
8     dorian_finney-smith
9            shake_milton
10           lebron_james
11               alex_len
12          rui_hachimura
13         jordan_goodwin
14           trey_jemison
15               luka_don
16        markieff_morris
Name: PLAYERCODE, dtype: object

In [26]:
drop = ["DISPLAY_LAST_COMMA_FIRST",'LAST_AFFILIATION',"ROSTERSTATUS", "GAMES_PLAYED_CURRENT_SEASON_FLAG","TEAM_NAME","TEAM_ABBREVIATION","TEAM_CODE","TEAM_CITY","PLAYERCODE","DLEAGUE_FLAG","NBA_FLAG","GAMES_PLAYED_FLAG","DRAFT_ROUND","GREATEST_75_FLAG"]

In [27]:
LAL_current_roster.drop(columns=drop, inplace=True)

In [28]:
LAL_current_roster.head()

,PERSON_ID,FIRST_NAME,LAST_NAME,DISPLAY_FIRST_LAST,DISPLAY_FI_LAST,PLAYER_SLUG,BIRTHDATE,SCHOOL,COUNTRY,HEIGHT,WEIGHT,SEASON_EXP,JERSEY,POSITION,TEAM_ID,FROM_YEAR,TO_YEAR,DRAFT_YEAR,DRAFT_NUMBER
0,1629020,Jarred,Vanderbilt,Jarred Vanderbilt,J. Vanderbilt,jarred-vanderbilt,1999-04-03T00:00:00,Kentucky,USA,6-8,214,7,2,Forward,1610612747,2018,2025,2018,41
1,1642261,Dalton,Knecht,Dalton Knecht,D. Knecht,dalton-knecht,2001-04-19T00:00:00,Tennessee,USA,6-6,215,1,4,Forward,1610612747,2024,2025,2024,17
2,1629216,Gabe,Vincent,Gabe Vincent,G. Vincent,gabe-vincent,1996-06-14T00:00:00,California-Santa Barbara,USA,6-2,200,6,7,Guard,1610612747,2019,2025,Undrafted,Undrafted
3,1642355,Bronny,James,Bronny James,B. James,bronny-james,2004-10-06T00:00:00,Southern California,USA,6-2,210,1,9,Guard,1610612747,2024,2025,2024,55
4,1631132,Christian,Koloko,Christian Koloko,C. Koloko,christian-koloko,2000-06-20T00:00:00,Arizona,Cameroon,6-11,225,3,10,Center,1610612747,2022,2025,2022,33


In [29]:
LAL_current_roster.columns

Index(['PERSON_ID', 'FIRST_NAME', 'LAST_NAME', 'DISPLAY_FIRST_LAST',
       'DISPLAY_FI_LAST', 'PLAYER_SLUG', 'BIRTHDATE', 'SCHOOL', 'COUNTRY',
       'HEIGHT', 'WEIGHT', 'SEASON_EXP', 'JERSEY', 'POSITION', 'TEAM_ID',
       'FROM_YEAR', 'TO_YEAR', 'DRAFT_YEAR', 'DRAFT_NUMBER'],
      dtype='object')

In [30]:
def get_roster_stats_by_team(teamAbrev):
    drop = ["PLAYER_SLUG","DISPLAY_FI_LAST","DISPLAY_LAST_COMMA_FIRST",'LAST_AFFILIATION',"ROSTERSTATUS", "GAMES_PLAYED_CURRENT_SEASON_FLAG","TEAM_NAME","TEAM_ABBREVIATION","TEAM_CODE","TEAM_CITY","PLAYERCODE","DLEAGUE_FLAG","NBA_FLAG","GAMES_PLAYED_FLAG","DRAFT_ROUND","GREATEST_75_FLAG"]
    team_roster = get_team_roster_per_season(teamAbrev,'2024-25')["PLAYER_ID"].tolist()
    team_current_roster = get_roster_data(team_roster)
    team_current_roster.drop(columns=drop, inplace=True)
    
    team_current_roster.to_json(f'{teamAbrev}_roster.json', orient = 'split', compression = 'infer', index = 'true')

    return team_current_roster
    
    

In [31]:
get_roster_stats_by_team("LAL")

,PERSON_ID,FIRST_NAME,LAST_NAME,DISPLAY_FIRST_LAST,BIRTHDATE,SCHOOL,COUNTRY,HEIGHT,WEIGHT,SEASON_EXP,JERSEY,POSITION,TEAM_ID,FROM_YEAR,TO_YEAR,DRAFT_YEAR,DRAFT_NUMBER
0,1629020,Jarred,Vanderbilt,Jarred Vanderbilt,1999-04-03T00:00:00,Kentucky,USA,6-8,214,7,2,Forward,1610612747,2018,2025,2018,41
1,1642261,Dalton,Knecht,Dalton Knecht,2001-04-19T00:00:00,Tennessee,USA,6-6,215,1,4,Forward,1610612747,2024,2025,2024,17
2,1629216,Gabe,Vincent,Gabe Vincent,1996-06-14T00:00:00,California-Santa Barbara,USA,6-2,200,6,7,Guard,1610612747,2019,2025,Undrafted,Undrafted
3,1642355,Bronny,James,Bronny James,2004-10-06T00:00:00,Southern California,USA,6-2,210,1,9,Guard,1610612747,2024,2025,2024,55
4,1631132,Christian,Koloko,Christian Koloko,2000-06-20T00:00:00,Arizona,Cameroon,6-11,225,3,10,Center,1610612747,2022,2025,2022,33
5,1629637,Jaxson,Hayes,Jaxson Hayes,2000-05-23T00:00:00,Texas,USA,7-0,220,6,11,Center-Forward,1610612747,2019,2025,2019,8
6,1628467,Maxi,Kleber,Maxi Kleber,1992-01-29T00:00:00,Bayern Munich,Germany,6-10,240,8,14,Forward,1610612747,2017,2025,Undrafted,Undrafted
7,1630559,Austin,Reaves,Austin Reaves,1998-05-29T00:00:00,Oklahoma,USA,6-5,197,4,15,Guard,1610612747,2021,2025,Undrafted,Undrafted
8,1627827,Dorian,Finney-Smith,Dorian Finney-Smith,1993-05-04T00:00:00,Florida,USA,6-7,220,9,2,Forward,1610612745,2016,2025,Undrafted,Undrafted
9,1629003,Shake,Milton,Shake Milton,1996-09-26T00:00:00,Southern Methodist,USA,6-5,205,8,20,Guard,1610612747,2018,2024,2018,54


In [32]:
get_roster_stats_by_team("GSW")

,PERSON_ID,FIRST_NAME,LAST_NAME,DISPLAY_FIRST_LAST,BIRTHDATE,SCHOOL,COUNTRY,HEIGHT,WEIGHT,SEASON_EXP,JERSEY,POSITION,TEAM_ID,FROM_YEAR,TO_YEAR,DRAFT_YEAR,DRAFT_NUMBER
0,1627780,Gary,Payton II,Gary Payton II,1992-12-01T00:00:00,Oregon State,USA,6-2,195,9,0,Guard,1610612744,2016,2025,Undrafted,Undrafted
1,1630228,Jonathan,Kuminga,Jonathan Kuminga,2002-10-06T00:00:00,NBA G League Ignite,DRC,6-7,225,4,1,Forward,1610612744,2021,2025,2021,7
2,1642379,Taran,Armstrong,Taran Armstrong,2002-01-15T00:00:00,"Cairns, Australia",Australia,6-5,190,1,1,Guard,1610612744,2024,2024,Undrafted,Undrafted
3,1641764,Brandin,Podziemski,Brandin Podziemski,2003-02-25T00:00:00,Santa Clara,USA,6-4,205,2,2,Guard,1610612744,2023,2025,2023,19
4,1630541,Moses,Moody,Moses Moody,2002-05-31T00:00:00,Arkansas,USA,6-5,211,4,4,Guard,1610612744,2021,2025,2021,14
5,1626172,Kevon,Looney,Kevon Looney,1996-02-06T00:00:00,UCLA,USA,6-9,222,10,55,Forward,1610612740,2015,2025,2015,30
6,1627741,Buddy,Hield,Buddy Hield,1992-12-17T00:00:00,Oklahoma,Bahamas,6-4,220,9,7,Guard,1610612744,2016,2025,2016,6
7,202710,Jimmy,Butler III,Jimmy Butler III,1989-09-14T00:00:00,Marquette,USA,6-6,230,14,10,Forward,1610612744,2011,2025,2011,30
8,1630296,Braxton,Key,Braxton Key,1997-02-14T00:00:00,Virginia,USA,6-8,225,5,12,Forward,1610612744,2021,2024,Undrafted,Undrafted
9,1630611,Gui,Santos,Gui Santos,2002-06-22T00:00:00,Minas,Brazil,6-7,185,2,15,Forward,1610612744,2023,2025,2022,55


In [33]:
get_roster_stats_by_team("DEN")

,PERSON_ID,FIRST_NAME,LAST_NAME,DISPLAY_FIRST_LAST,BIRTHDATE,SCHOOL,COUNTRY,HEIGHT,WEIGHT,SEASON_EXP,JERSEY,POSITION,TEAM_ID,FROM_YEAR,TO_YEAR,DRAFT_YEAR,DRAFT_NUMBER
0,1631128,Christian,Braun,Christian Braun,2001-04-17T00:00:00,Kansas,USA,6-6,220,3,0,Guard,1610612743,2022,2025,2022,21
1,1629008,Michael,Porter Jr.,Michael Porter Jr.,1998-06-29T00:00:00,Missouri,USA,6-10,218,6,17,Forward,1610612751,2018,2025,2018,14
2,1631124,Julian,Strawther,Julian Strawther,2002-04-18T00:00:00,Gonzaga,USA,6-6,205,2,3,Guard,1610612743,2023,2025,2023,29
3,201566,Russell,Westbrook,Russell Westbrook,1988-11-12T00:00:00,UCLA,USA,6-4,200,17,18,Guard,1610612758,2008,2025,2008,4
4,1641816,Hunter,Tyson,Hunter Tyson,2000-06-13T00:00:00,Clemson,USA,6-8,215,2,5,Forward,1610612743,2023,2025,2023,37
5,201599,DeAndre,Jordan,DeAndre Jordan,1988-07-21T00:00:00,Texas A&M,USA,6-11,265,17,6,Center,1610612740,2008,2025,2008,35
6,1631212,Peyton,Watson,Peyton Watson,2002-09-11T00:00:00,UCLA,USA,6-8,200,3,8,Guard,1610612743,2022,2025,2022,30
7,203967,Dario,Šarić,Dario Šarić,1994-04-08T00:00:00,Anadolu Efes,Croatia,6-10,225,9,9,Forward-Center,1610612758,2016,2025,2014,12
8,1641790,PJ,Hall,PJ Hall,2002-02-21T00:00:00,Clemson,USA,6-8,245,1,16,Center,1610612763,2024,2025,Undrafted,Undrafted
9,1641747,DaRon,Holmes II,DaRon Holmes II,2002-08-15T00:00:00,Dayton,USA,6-9,225,0,14,Forward,1610612743,2024,2025,2024,22


In [34]:
get_roster_stats_by_team("DAL")

,PERSON_ID,FIRST_NAME,LAST_NAME,DISPLAY_FIRST_LAST,BIRTHDATE,SCHOOL,COUNTRY,HEIGHT,WEIGHT,SEASON_EXP,JERSEY,POSITION,TEAM_ID,FROM_YEAR,TO_YEAR,DRAFT_YEAR,DRAFT_NUMBER
0,203957,Danté,Exum,Danté Exum,1995-07-13T00:00:00,Australian Institute of Sport,Australia,6-5,214,9,0,Guard,1610612742,2014,2025,2014,5
1,1631108,Max,Christie,Max Christie,2003-02-10T00:00:00,Michigan State,USA,6-5,190,3,00,Guard,1610612742,2022,2025,2022,35
2,1630702,Jaden,Hardy,Jaden Hardy,2002-07-05T00:00:00,NBA G League Ignite,USA,6-3,198,3,1,Guard,1610612742,2021,2025,2022,37
3,1641726,Dereck,Lively II,Dereck Lively II,2004-02-12T00:00:00,Duke,USA,7-1,230,2,2,Center,1610612742,2023,2025,2023,12
4,203076,Anthony,Davis,Anthony Davis,1993-03-11T00:00:00,Kentucky,USA,6-10,253,13,3,Forward-Center,1610612742,2012,2025,2012,1
5,203939,Dwight,Powell,Dwight Powell,1991-07-20T00:00:00,Stanford,Canada,6-10,240,11,7,Forward-Center,1610612742,2014,2025,2014,45
6,1641765,Olivier-Maxence,Prosper,Olivier-Maxence Prosper,2002-07-03T00:00:00,Marquette,Canada,6-7,230,2,18,Forward,1610612763,2023,2025,2023,24
7,1630314,Brandon,Williams,Brandon Williams,1999-11-22T00:00:00,Arizona,USA,6-1,190,3,10,Guard,1610612742,2021,2025,Undrafted,Undrafted
8,202681,Kyrie,Irving,Kyrie Irving,1992-03-23T00:00:00,Duke,Australia,6-2,195,14,11,Guard,1610612742,2011,2025,2011,1
9,1630230,Naji,Marshall,Naji Marshall,1998-01-24T00:00:00,Xavier,USA,6-6,220,5,13,Forward,1610612742,2020,2025,Undrafted,Undrafted
